In [1]:
# !pip install pandas tqdm transformers accelerate bitsandbytes

In [2]:
import re
import os
import pandas as pd
from tqdm import tqdm

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [3]:
test = pd.read_csv('../data/test.csv')
test

,ID,Question
0,TEST_000,금융산업의 이해와 관련하여 금융투자업의 구분에 해당하지 않는 것은?\n1 소비자금융...
1,TEST_001,위험 관리 계획 수립 시 고려해야 할 요소로 적절하지 않은 것은?\n1 수행인력\n...
2,TEST_002,관리체계 수립 및 운영'의 '정책 수립' 단계에서 가장 중요한 요소는 무엇인가?\n...
3,TEST_003,재해 복구 계획 수립 시 고려해야 할 요소로 옳지 않은 것은?\n1 복구 절차 수립...
4,TEST_004,트로이 목마(Trojan) 기반 원격제어 악성코드(RAT)의 특징과 주요 탐지 지표...
...,...,...
510,TEST_510,"""정보보호최고책임자""의 임명에 관한 설명으로 옳지 않은 것은?\n1 정보보호최고책임..."
511,TEST_511,IPv6 주소 체계의 주요 특징으로 옳지 않은 것은?\n1 NAT 필요성 감소\n2...
512,TEST_512,하이브리드 위협에 대한 설명으로 가장 적절한 것은?\n1 사이버 공간에서만 발생하는...
513,TEST_513,전자금융거래법의 주요 목적 중 하나는 무엇인가?\n1 전자금융거래의 비대면성 강화\...


In [4]:
test = test.iloc[[492, 446, 362, 111, 121, 162, 434, 4, 7, 82, 101, 102, 126, 131, 182, 458], :]
# test = test.iloc[:20, :]1

In [5]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [6]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                # "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                "아래 질문에 대해 정답의 핵심 키워드와 의미를 모두 포함하여 3문장 이내로 간결하게 답변하세요. 군더더기 없이 요점만 명확하게 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

In [7]:
# -*- coding: utf-8 -*-
# 멀티법령 RAG 전처리/인덱싱 (PIPA, 신용정보법, 전자서명법, 정보통신망법)

import os, re, json, math
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional

import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceEmbeddings
import faiss

# ===== 사용자 환경 상수 =====\
EMB_MODEL = "jhgan/ko-sroberta-multitask"   # 경량/호환성 위주
# EMB_MODEL = "intfloat/multilingual-e5-base"
LLM_ID = "nlpai-lab/KULLM3"
CHUNK_TOKENS = 600
CHUNK_OVERLAP = 32
CTX_TOKEN_BUDGET = 600
TOP_K = 4
SEED = 42
torch.manual_seed(SEED)

# ===== 법령 설정: 파일 경로 + 제거/정규식 패턴 =====
LAW_CONFIG = {
    # 개인정보 보호법
    "pipa": {
        "law_name": "개인정보 보호법",
        "pdf_path": "../data/개인정보 보호법(법률)(제19234호)(20250313).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*개인정보\s*보호법',
            r'법제처\s+\d+\s+국가법령정보센터',
            r'국가법령정보센터\s*개인정보\s*보호법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>',         # <개정 …>, <신설 …>
            r'\[[^\]]+\]',      # [본조신설 …]
        ],
    },
    # 신용정보의 이용 및 보호에 관한 법률
    "ciupa": {
        "law_name": "신용정보법",
        "pdf_path": "../data/신용정보의 이용 및 보호에 관한 법률(법률)(제20304호)(20240814).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*신용정보.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자서명법
    "es_act": {
        "law_name": "전자서명법",
        "pdf_path": "../data/전자서명법(법률)(제18479호)(20221020).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자서명법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 정보통신망 이용촉진 및 정보보호 등에 관한 법률
    "icn_act": {
        "law_name": "정보통신망법",
        "pdf_path": "../data/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20678호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*정보통신망.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융거래법
    "eft_act": {
        "law_name": "전자금융거래법",
        "pdf_path": "../data/전자금융거래법(법률)(제19734호)(20240915).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 전자금융감독규정
    "rs_act": {
        "law_name": "전자금융감독규정",
        "pdf_path": "../data/전자금융감독규정(금융위원회고시)(제2025-4호)(20250205).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*전자금융거래.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
    # 자본시장법
    "fis_act": {
        "law_name": "자본시장법",
        "pdf_path": "../data/자본시장과 금융투자업에 관한 법률(법률)(제20718호)(20250722).pdf",
        "drop_patterns": [
            r'법제처\s+\d+\s+국가법령정보센터\s*자본시장.*법',
            r'법제처|국가법령정보센터',
            r'<[^>]+>', r'\[[^\]]+\]',
        ],
    },
}

In [8]:
# ===== 토크나이저: 토큰 길이 계산/청킹 =====
llm_tokenizer = AutoTokenizer.from_pretrained(LLM_ID)
if llm_tokenizer.pad_token is None:
    llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right"

def token_len(s: str) -> int:
    return len(llm_tokenizer(s, add_special_tokens=False)["input_ids"])

def split_sentences_ko(text: str) -> List[str]:
    # 고정폭 lookbehind: '다.' 또는 일반 종결부호
    text = re.sub(r'\s+', ' ', text).strip()
    if not text:
        return []
    return re.split(r'(?<=다\.)\s+|(?<=[.?!。！？])\s+', text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# ===== 공통 정제 =====
def normalize_common(text: str) -> str:
    # 한자 제거
    text = re.sub(r'[\u4e00-\u9fff]', '', text)
    # circled numbers → (n)
    circled = '①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'
    for idx, c in enumerate(circled, 1):
        text = text.replace(c, f'({idx})')
    # 공백/빈 괄호 정리
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\(\s*\)', '', text)
    return text

def clean_text_by_config(text: str, drop_patterns: List[str]) -> str:
    for pat in drop_patterns:
        text = re.sub(pat, '', text)
    return normalize_common(text)

# ===== 조문 단위 분리 =====
ARTICLE_HEADER_PATTERN = r'(제\d+조(?:의\d+)?\([^)]+\))'  # 제X조(제목) / 제X조의Y(제목)

def split_articles(raw_text: str) -> List[Tuple[str, str, str]]:
    parts = re.split(ARTICLE_HEADER_PATTERN, raw_text)
    out = []
    for i in range(1, len(parts), 2):
        header = parts[i]
        body = (parts[i+1] if i+1 < len(parts) else "").strip().replace("\n", " ")
        m = re.match(r'(제\d+조(?:의\d+)?)[(]([^)]+)[)]', header)
        if not m:
            continue
        article_id = m.group(1)          # 제xx조 / 제xx조의y
        article_title = m.group(2)       # (제목)
        out.append((article_id, article_title, body))
    return out

def chunk_article(article_body: str, header: str) -> List[str]:
    prefix = header.strip() + "\n"
    sents = split_sentences_ko(article_body) or [article_body]
    chunks, cur, cur_toks = [], [], token_len(prefix)
    for s in sents:
        tl = token_len(s)
        if cur_toks + tl > CHUNK_TOKENS and cur:
            chunks.append(prefix + " ".join(cur))
            # overlap: 마지막 문장 유지
            keep = cur[-1] if CHUNK_OVERLAP > 0 and cur else ""
            cur = [keep] if keep else []
            cur_toks = token_len(prefix) + (token_len(keep) if keep else 0)
        cur.append(s); cur_toks += tl
    if cur:
        chunks.append(prefix + " ".join(cur))
    return chunks


In [10]:
# ===== 데이터 클래스 =====
@dataclass
class LawDoc:
    text: str
    meta: Dict

# ===== 임베딩 =====
embeddings = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
    encode_kwargs={
        "normalize_embeddings": True,
        "batch_size": 128,
        "convert_to_numpy": True,
        "convert_to_tensor": False
    }
)

# ===== 인덱스 (FAISS HNSW) =====
def build_faiss_hnsw(vectors: np.ndarray, m: int = 32, ef_search: int = 32) -> faiss.IndexHNSWFlat:
    dim = vectors.shape[1]
    idx = faiss.IndexHNSWFlat(dim, m)
    idx.hnsw.efSearch = ef_search
    idx.add(vectors.astype(np.float32))
    return idx

# ===== 파이프라인: 1) 로드 → 2) 정제 → 3) 조문분리 → 4) 청킹 → 5) 임베딩/인덱스
def load_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    text = ""
    for p in reader.pages:
        t = p.extract_text() or ""
        text += t + "\n"
    return text

def preprocess_law(law_id: str, cfg: Dict) -> List[LawDoc]:
    raw = load_pdf_text(cfg["pdf_path"])
    cleaned = clean_text_by_config(raw, cfg["drop_patterns"])
    articles = split_articles(cleaned)

    docs: List[LawDoc] = []
    # 시행일 파싱(있으면): [시행 YYYY. M. D.]가 머리말에 있는 경우가 많으나 PDF마다 다름 → 필요 시 별도 파서
    effective_date = None  # 필요 시 추출 로직 추가

    for article_id, title, body in articles:
        header = f'{cfg["law_name"]} {article_id}({title})'
        chunks = chunk_article(body, header)
        for ch in chunks:
            meta = {
                "law_id": law_id,
                "law_name": cfg["law_name"],
                "article_id": article_id,
                "article_title": title,
                "effective_date": effective_date,   # None 가능
                "tok_len": token_len(ch),
                "source_uri": cfg.get("pdf_path"),
                "version": None
            }
            docs.append(LawDoc(text=ch, meta=meta))
    return docs

def build_indices(all_docs: List[LawDoc]):
    # (a) 법령별 인덱스
    per_law_docs: Dict[str, List[LawDoc]] = {}
    for d in all_docs:
        per_law_docs.setdefault(d.meta["law_id"], []).append(d)

    indices = {}
    for law_id, docs in per_law_docs.items():
        mat = np.array(embeddings.embed_documents([d.text for d in docs]), dtype=np.float32)
        indices[f"faiss_hnsw_{law_id}"] = {
            "index": build_faiss_hnsw(mat, m=32, ef_search=32),
            "docs": docs
        }

    # (b) 글로벌 인덱스
    mat_all = np.array(embeddings.embed_documents([d.text for d in all_docs]), dtype=np.float32)
    indices["faiss_hnsw_all"] = {
        "index": build_faiss_hnsw(mat_all, m=32, ef_search=32),
        "docs": all_docs
    }
    return indices

/tmp/ipykernel_9059/3198458753.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
all_docs: List[LawDoc] = []
for law_id, cfg in LAW_CONFIG.items():
    if not os.path.exists(cfg["pdf_path"]):
        print(f"[WARN] PDF not found: {cfg['pdf_path']}")
        continue
    docs = preprocess_law(law_id, cfg)
    all_docs.extend(docs)
    print(f"[OK] {cfg['law_name']} → chunks: {len(docs)}")

indices = build_indices(all_docs)
print("[OK] built indices:", list(indices.keys()))

[OK] 개인정보 보호법 → chunks: 217
[OK] 신용정보법 → chunks: 181
[OK] 전자서명법 → chunks: 29
[OK] 정보통신망법 → chunks: 151
[OK] 전자금융거래법 → chunks: 100
[OK] 전자금융감독규정 → chunks: 146
[OK] 자본시장법 → chunks: 1092
[OK] built indices: ['faiss_hnsw_pipa', 'faiss_hnsw_ciupa', 'faiss_hnsw_es_act', 'faiss_hnsw_icn_act', 'faiss_hnsw_eft_act', 'faiss_hnsw_rs_act', 'faiss_hnsw_fis_act', 'faiss_hnsw_all']


In [12]:
# ===== 라우팅 규칙 =====
ARTICLE_PTRN = re.compile(r"제\d+조(?:의\d+)?")  # 조문 표기
LAW_HINTS = {
    "pipa": ("개인정보", "개인 정보", "개인정보보호법"),
    "ciupa": ("신용정보",  "신용정보법"),
    "es_act": ("전자서명", "전자서명법"),
    "icn_act": ("정보통신망", "통신망", "정보통신망법"),
    "eft_act": ("전자금융", "금융거래", "전자금융거래법"),
    "rs_act": ("전자금융감독", "금융감독", "전자금융감독규정"),
    "fis_act": ("자본", "자본시장", "금융투자", "채권", "자본시장법", "금융투자업", "투자"),
}

def detect_law_id(query: str) -> Optional[str]:
    q = query.lower()
    for law_id, kws in LAW_HINTS.items():
        if any(kw.lower() in q for kw in kws):
            return law_id
    return None

def route_is_domain(query: str) -> bool:
    # 법/금융/보안 도메인 간단 라우터 (검색 여부 판단용)
    domain_kws = ("법", "조(", "과징금", "처벌", "보안", "침해", "금융", "개인정보", "신용정보", "전자서명", "정보통신", "전자금융", "금융감독", "자본", "자본시장", "금융투자", "채권", "투자")
    q = query.lower()
    return any(kw in q for kw in domain_kws) or bool(ARTICLE_PTRN.search(query))

def choose_index(indices: dict, query: str):
    """
    1) 질의에서 법령 단서 -> 해당 법 인덱스 우선
    2) 조문 패턴만 있거나 단서가 불분명 -> 글로벌 인덱스
    3) 아무 단서도 없으면 None (베이스모델 경로)
    """
    law_id = detect_law_id(query)
    if law_id:
        key = f"faiss_hnsw_{law_id}"
        if key in indices:
            return indices[key]  # {"index": ..., "docs": ...}
    # 법령 단서 없지만 도메인성/조문 표기는 있는 경우 글로벌
    if route_is_domain(query) and "faiss_hnsw_all" in indices:
        return indices["faiss_hnsw_all"]
    return None  # 베이스모델 직행


In [13]:
# ===== 검색 with 점수 (해당 인덱스에서) =====
def faiss_search_with_scores_from_index(index_entry: dict, query: str, top_k: int = TOP_K):
    # embeddings는 상위 스코프에서 로드되었다고 가정
    qv = np.array(embeddings.embed_query(query), dtype=np.float32).reshape(1, -1)
    D, I = index_entry["index"].search(qv, top_k)     # L2 거리 (정규화 벡터 가정)
    cos = 1.0 - (D[0] / 2.0)                          # L2 -> cosine
    out = []
    docs = index_entry["docs"]
    for idx, i in enumerate(I[0]):
        ii = int(i)
        if ii >= 0:
            out.append((docs[ii], float(cos[idx])))
    return out  # [(LawDoc, cosine), ...]

# ===== 컨텍스트 패킹 (질문 주신 코드 재사용 + tok_len 캐시) =====
def pack_context(docs_in, token_budget=CTX_TOKEN_BUDGET):
    acc, used = [], 0
    for d in docs_in:
        tl = d.meta.get("tok_len", None)
        if tl is None:
            tl = token_len(d.text); d.meta["tok_len"] = tl
        if used + tl <= token_budget:
            acc.append(d.text); used += tl
        else:
            remain = token_budget - used
            if remain > 50:
                ids = llm_tokenizer(d.text, add_special_tokens=False)["input_ids"][:remain]
                acc.append(llm_tokenizer.decode(ids))
            break
    return "\n\n".join(acc)


In [14]:
# ===== 프롬프트 =====
def chat_prompt(system, user):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]
    return llm_tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

def build_messages(query: str, use_context: bool, context: Optional[str]):
    # 시스템 지시문
    system = "당신은 금융/보안 QA 도우미입니다. 포맷을 엄격히 지키세요."
    # 유저 메시지
    if is_multiple_choice(query):
        question, options = extract_question_and_choices(query)
        if use_context:
            user = (
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
            )
        else:
            user = (
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
            )
    else:
        rules = (
            "아래 질문에 대해 **사실에 근거한 간결한 답변**을 작성하세요.\n\n"
            "규칙:\n"
            "1. 답변은 2~3문장 이내로 작성합니다. 장황한 서론, 결론 문구는 쓰지 않습니다.\n"
            "2. 불확실할 경우 '알 수 없습니다'라고 답하고 생성을 종료합니다.\n"
            "3. 특수문자 없이 오로지 한글과 숫자로만 대답합니다.\n"
        )
        if use_context:
            user = (
                "아래 컨텍스트를 우선 사용해 정확히 답하세요. 불충분하면 아는 범위에서만 간결히 답하세요.\n\n"
                f"=== 컨텍스트 ===\n{context}\n=== 끝 ===\n"
                f"{rules}\n질문: {query}\n답변:"
            )
        else:
            user = f"{rules}\n질문: {query}\n답변:"

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": user},
    ]
    return messages


# ===== 생성 토큰 상한 =====
def dynamic_max_new_tokens(question: str) -> int:
    lines = [ln.strip() for ln in question.split("\n") if ln.strip()]
    opt_cnt = sum(bool(re.match(r"^\d+(\s|[.)])", ln)) for ln in lines)
    return 96 if opt_cnt >= 2 else 192

def generate_with_llama(messages: List[Dict], max_new_tokens: int) -> str:
    """
    llama-cpp chat completion. 모델 메타데이터(chat_format)가 있으면 자동 템플릿 적용.
    """
    res = llm_model.create_chat_completion(
        messages=messages,
        temperature=0.2,
        top_p=0.9,
        max_tokens=max_new_tokens,
        stop=["<|eot_id|>"],  # Llama-3 계열 종료 토큰
    )
    return res["choices"][0]["message"]["content"].strip()

# ---------------- 메인 RAG 생성 ----------------
def generate_answer_with_indices(query: str, indices: dict) -> str:
    # 비도메인 → RAG 미사용
    if not route_is_domain(query):
        messages = build_messages(query, use_context=False, context=None)
        return generate_with_llama(messages, dynamic_max_new_tokens(query))

    # 인덱스 선택
    idx_entry = choose_index(indices, query)

    # 인덱스가 없으면 베이스모델
    if idx_entry is None:
        messages = build_messages(query, use_context=False, context=None)
        return generate_with_llama(messages, dynamic_max_new_tokens(query))

    # 검색
    scored = faiss_search_with_scores_from_index(idx_entry, query, top_k=TOP_K)
    best_cos = max((s for _, s in scored), default=0.0)
    # 임계치
    THRESH = 0.70
    use_context = best_cos >= THRESH and len(scored) > 0
    context = pack_context([d for d, _ in scored], token_budget=CTX_TOKEN_BUDGET) if use_context else None

    # 생성
    messages = build_messages(query, use_context, context)
    return generate_with_llama(messages, dynamic_max_new_tokens(query))


In [5]:
import os; print(os.environ.get("LD_LIBRARY_PATH","<empty>"))
os.environ["LD_LIBRARY_PATH"] = "/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:" + os.environ.get("LD_LIBRARY_PATH","/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib/libcudart.so.12")

/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [6]:
import os, sysconfig, glob, ctypes

# 1) site-packages에서 nvidia CUDA/BLAS의 lib 디렉토리 자동 탐색
paths = []
for key in ("purelib", "platlib"):
    base = sysconfig.get_paths().get(key)
    if not base: 
        continue
    paths += [p for p in glob.glob(os.path.join(base, "nvidia", "*", "lib")) if os.path.isdir(p)]

if not paths:
    raise RuntimeError("nvidia CUDA/BLAS wheel의 lib 폴더를 찾지 못했습니다. pip 설치 상태를 다시 확인하세요.")

# 2) LD_LIBRARY_PATH에 주입 (현재 프로세스에 즉시 반영)
os.environ["LD_LIBRARY_PATH"] = ":".join(paths) + ":" + os.environ.get("LD_LIBRARY_PATH","")

# 3) 실제 so 로딩이 되는지 사전 점검 (여기서 실패하면 경로 문제)
for name in ("libcudart.so.12", "libcublas.so.12"):
    try:
        ctypes.CDLL(name)
        print(f"OK: {name} loaded")
    except OSError as e:
        raise RuntimeError(f"FAIL: {name} not found -> {e}\nLD_LIBRARY_PATH={os.environ['LD_LIBRARY_PATH']}")

print("CUDA runtime and cuBLAS visible ✓")


RuntimeError: FAIL: libcudart.so.12 not found -> libcudart.so.12: cannot open shared object file: No such file or directory
LD_LIBRARY_PATH=/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cuda_runtime/lib:/usr/local/lib/python3.10/dist-packages/nvidia/cublas/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64

In [1]:
from llama_cpp import Llama
print("OK: imported Llama")

RuntimeError: Failed to load shared library '/usr/local/lib/python3.10/dist-packages/llama_cpp/lib/libllama.so': libcudart.so.12: cannot open shared object file: No such file or directory

In [15]:
from llama_cpp import Llama

# ---------------- LLM (llama-cpp) GPU 로드 ----------------
GGUF_PATH = "../models/kullm3/KULLM3-Q5_K_M.gguf"
assert os.path.exists(GGUF_PATH), f"GGUF model not found: {GGUF_PATH}"

USE_GPU = torch.cuda.is_available()
if not USE_GPU:
    print("[WARN] CUDA가 감지되지 않았습니다. 현재 환경에서는 CPU로 동작합니다.")
    print("       CUDA 휠(예: llama-cpp-python-cu118)로 재설치해야 GPU 사용이 가능합니다.")

# vCPU 수에 맞게 스레드 설정
N_THREADS = max(2, (os.cpu_count() or 6))
# VRAM 넉넉한 4090 기준 배치 크게, CPU일 때는 작게
N_BATCH = 512 if USE_GPU else 128
# 가능한 모든 레이어를 GPU로: -1, CPU일 경우 0
N_GPU_LAYERS = -1 if USE_GPU else 0

# 참고: 최신 llama-cpp-python에서는 offload_kqv, main_gpu, tensor_split 등을 지원
llm_model = Llama(
    model_path=GGUF_PATH,
    n_ctx=4096,
    n_batch=N_BATCH,          # GPU에서 큰 배치로 처리량↑
    n_threads=N_THREADS,      # 토큰화/호스트 연산 스레드
    n_gpu_layers=N_GPU_LAYERS,# 가능한 레이어를 GPU로 오프로딩(-1=전부)
    main_gpu=0,               # 단일 GPU 사용 시 0
    offload_kqv=True if USE_GPU else False,  # K/Q/V 연산도 GPU로
    seed=SEED,
    logits_all=False,
    use_mmap=True,            # 메모리 매핑(속도/메모리 측면 유리)
    use_mlock=False,          # 메모리 고정(권한/메모리 여유에 따라 조정)
    f16_kv=True,
    verbose=True,
)

print(f"[OK] llama-cpp loaded. GPU: {USE_GPU}, n_gpu_layers={N_GPU_LAYERS}, n_batch={N_BATCH}, n_threads={N_THREADS}")


RuntimeError: Failed to load shared library '/usr/local/lib/python3.10/dist-packages/llama_cpp/lib/libllama.so': libcudart.so.12: cannot open shared object file: No such file or directory

In [ ]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text


In [ ]:
def strip_explanations(answer: str) -> str:
    """
    키워드 뒤 설명을 제거하고 키워드만 남김
    - '키워드: 설명' → '키워드'
    - '키워드 ... 설명문' → '키워드'
    """
    lines = answer.split("\n")
    cleaned = []
    for line in lines:
        # "키워드: 설명" → "키워드"
        m = re.match(r"^\s*[-0-9.]*\s*([^\:]+)", line)
        if m:
            keyword = m.group(1).strip()
            cleaned.append(keyword)
    return "\n".join(cleaned)

In [ ]:
test = test.iloc[[0], :]
test

In [ ]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    print("#################### Question ###########################")
    print(q)
    ans = generate_answer_with_indices(q, indices)
    pred_answer = extract_answer_only(ans, original_question=q)
    print("#################### Answer ###########################")
    print(strip_explanations(pred_answer))
    preds.append(pred_answer)

In [ ]:
# sample_submission = pd.read_csv('../submission/sample_submission.csv')
# sample_submission['Answer'] = preds
# sample_submission.to_csv('../submission/gen_v4.03_submission.csv', index=False, encoding='utf-8-sig')